# detect differences in path count table

compare path count tables from

- before: as of [connectivity-search-backend/pull/41](https://github.com/greenelab/connectivity-search-backend/pull/41#issuecomment-488054789)
- after: as of [connectivity-search-backend/pull/79](https://github.com/greenelab/connectivity-search-backend/pull/79#issuecomment-759114378)

In [9]:
import pandas
import psycopg2

In [15]:
sql = '''\
SELECT metapath_id, COUNT(*) AS path_count_rows
FROM dj_hetmech_app_pathcount
GROUP BY metapath_id
'''

con_before = psycopg2.connect(
    host="search-db.het.io",
    port=5432,
    dbname="connectivity_db",
    user="read_only_user",
    password="tm8ut9uzqx7628swwkb9",
)
before_df = pandas.read_sql_query(con=con_before, sql=sql)

con_after = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="connectivity_db",
    user="dj_hetmech",
    password="not_secure",
)
after_df = pandas.read_sql_query(con=con_after, sql=sql)

In [27]:
after_df.head(2)

,metapath_id,path_count_rows
0,AdG,102240
1,AdGaD,203


In [28]:
before_df.head(2)

,metapath_id,path_count_rows
0,AdG,102240
1,AdGaD,203


In [41]:
both_df = (
    before_df
    .merge(after_df, how="outer", on="metapath_id", suffixes=("_before", "_after"))
    .convert_dtypes()
)
both_df.head(2)

,metapath_id,path_count_rows_before,path_count_rows_after
0,AdG,102240,102240
1,AdGaD,203,203


In [42]:
both_df.sum()

path_count_rows_before    166199174
path_count_rows_after     169602979
dtype: int64

In [43]:
both_df.query("path_count_rows_before != path_count_rows_after").head()

,metapath_id,path_count_rows_before,path_count_rows_after
637,CbGaD,1791,27498
641,CbGaDrD,192,87203
648,CbGbCpD,333,39723
649,CbGbCtD,689,60363
652,CbGcGaD,132,24363


In [48]:
len(both_df.query("path_count_rows_before != path_count_rows_after").metapath_id)

132

In [44]:
both_df.query("path_count_rows_before > path_count_rows_after")

,metapath_id,path_count_rows_before,path_count_rows_after
